In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
from tqdm import tqdm

tqdm.pandas()

In [ ]:
unique_id_header = "lib_plate_well"
lib_id = "mce"
plate_id_header = "mixed_location_plate1"

In [ ]:
lib_df = pd.read_csv("data/mce_library_first_metadata_raw.csv")

In [ ]:
lib_df.head()

In [ ]:
# lib_df[["WellLetter", "WellNumber"]] = lib_df["Plate Location"].str.split(expand=True)
# lib_df["WellLetter"] = [value[:1] for value in lib_df["Plate Location"]]
# lib_df["WellNumber"] = [int(value[1:]) for value in lib_df["Plate Location"]]
# lib_df.drop(columns=["WellNumber", "WellLetter"], inplace=True)
lib_df[["WellNumber", "WellLetter"]] = [[int(value[1:]),value[:1]] for value in lib_df["Plate Location"]]

In [ ]:
sorted_df = lib_df.sort_values(by=["RackCode", "WellLetter", "WellNumber"], ascending=[True, True, True])
sorted_df

In [ ]:
def next_well(well, letter, plate):
    well += 1
    if well > 24:
        well = 1

        # next letter
        letter = ord(letter) + 1
        if letter>80:
            letter = 65
            plate += 1

        letter = chr(letter)

    return well, letter, plate


plate = 1
well = 1
letter = "A"
original_letter = "A"

mixed_plates = []
mixed_wells = []

for index, row in sorted_df.iterrows():
    tmp_letter = row["WellLetter"]
    if tmp_letter is not original_letter:
        original_letter = tmp_letter
        well, letter, plate = next_well(well, letter, plate)

    mixed_plates.append(plate)
    mixed_wells.append("{}{}".format(letter, well))

sorted_df["MixedPlate"] = mixed_plates
sorted_df["MixedWell"] = mixed_wells
sorted_df["mixed_location_plate1"] = ["1D{}_{}".format(plate, well) for well, plate in zip(mixed_wells, mixed_plates)]
sorted_df["mixed_location_plate2"] = ["2D{}_{}".format(plate, well) for well, plate in zip(mixed_wells, mixed_plates)]
sorted_df["mixed_location_plate3"] = ["3D{}_{}".format(plate, well) for well, plate in zip(mixed_wells, mixed_plates)]
sorted_df[unique_id_header] = ["pluskal_{}_{}".format(lib_id, plate_id) for plate_id in sorted_df[plate_id_header]]

sorted_df

In [ ]:
sorted_df.to_csv("data/mce_library.tsv", sep="\t", index=False)

## 100+ additional compounds

In [ ]:
add_df = pd.read_csv("data/mce_library_additional_compounds_raw.csv")
add_df = add_df.drop(["Short Name (A & B Coulumn)", "Mixture plate name"], axis=1)

In [ ]:
add_df

In [ ]:
add_df[["WellNumber", "WellLetter"]] = [[int(value[1:]),value[:1]] for value in add_df["Plate Location"]]
sorted2_df = add_df.sort_values(by=["RackCode", "WellLetter", "WellNumber"], ascending=[True, True, True])
sorted2_df.rename(columns={"Diltuted Duplicate 1": "mixed_location_plate1", "Diltuted Duplicate 2": "mixed_location_plate2", "Diltuted Duplicate 3": "mixed_location_plate3" }, inplace=True)
sorted2_df[unique_id_header] = ["pluskal_{}_{}".format(lib_id, plate_id) for plate_id in sorted2_df[plate_id_header]]
sorted2_df


In [ ]:
sorted2_df.to_csv("data/mce_library_add_compounds.tsv", sep="\t", index=False)

In [ ]:
df = pd.read_csv("data/mce_library_cleaned.tsv", sep="\t")
cleaned_df = pd.read_csv("data/mce_library_add_compounds_cleaned.tsv", sep="\t")

In [ ]:
df

In [ ]:
cleaned_df

In [ ]:
df_all= pd.concat([df.set_index("Product Name"), cleaned_df.set_index("Product Name")])

In [ ]:
df_all

In [ ]:
df_all.drop_duplicates(["lib_plate_well", "Product Name"], keep=second)

In [ ]:
df = pd.read_csv("data/final_tables/mce_library_all_final_unique_id.tsv", sep="\t")

In [ ]:
df

In [ ]:
df["lib_plate_well_unique"] = df["lib_plate_well"] + "_"

In [ ]:
df

In [ ]:
df.to_csv("data/final_tables/nih_library_all_final_unique_id.tsv", sep="\t", index=False)

In [ ]:
df = pd.read_csv("data/nih/nih_library.tsv", sep="\t")
df

In [ ]:
df.drop_duplicates(["lib_plate_well"])

In [ ]:
df = pd.read_csv("data/final_tables/mce_library_all_final_unique_id.tsv", sep="\t")

In [ ]:
df

In [ ]:
df.loc[df["split_inchi_key"]=="RXGUTQNKCXHALN"]